# EDA: NBA Champions and Runner-Ups Since 1950
To be used after executing data_clean_up.ipynb file
## Importing required libraries

In [2]:
import pandas as pd
import numpy as np
import plotly.express as px
import chart_studio.plotly as py
import chart_studio
import plotly.graph_objects as go
from pathlib import Path
from string import ascii_lowercase
import os

In [60]:
# to upload graphs to your plotly profile
#chart_studio.tools.set_credentials_file(username='username',api_key='api-key')

## Select and load csv files produced by data_clean_up.ipynb file
### go to directory with files

In [10]:
# only use below if you need to go to file directory
#back_dir = os.path.normpath(os.getcwd() + os.sep + os.pardir)
#os.getcwd()
#os.chdir(back_dir)
#os.chdir('test')
#os.listdir()

'C:\\Users\\davil\\Desktop\\test'

## Get dataframes for EDA

In [11]:
#champions and runner-ups
df_champs = pd.read_csv('champs/champions_index.csv')
#regular season data, including mvp shares
df_rs_pergame = pd.read_csv('clean_reg_season/all_seasons_per_game_stats.csv')
df_rs_totals = pd.read_csv('clean_reg_season/all_seasons_totals_stats.csv')
df_rs_advanced = pd.read_csv('clean_reg_season/all_seasons_advanced_stats.csv')
df_mvps = pd.read_csv('clean_reg_season/all_seasons_nba_mvp.csv')
#playoffs data
df_po_pergame = pd.read_csv('clean_playoffs/all_seasons_per_game_stats.csv')
df_po_totals = pd.read_csv('clean_playoffs/all_seasons_totals_stats.csv')
df_po_advanced = pd.read_csv('clean_playoffs/all_seasons_advanced_stats.csv')
#player bios
df_bios = pd.read_csv('clean_bios/all_players_bios.csv')

### looking into all teams that made a NBA Finals appearance

In [12]:
nba_champs = df_champs[df_champs.Lg == 'NBA']
runnerups_counts = nba_champs.groupby('Runner-Up')['Runner-Up'].count().reset_index(name='counts')
runnerups_counts = pd.DataFrame(runnerups_counts)
runnerups_counts = runnerups_counts.rename(columns={'Runner-Up':'Tm'})
runnerups_counts['Type']='Runner-Ups'

runnerups_counts
champ_counts = nba_champs.groupby('Champion')['Champion'].count().reset_index(name='counts')
champ_counts = pd.DataFrame(champ_counts)
champ_counts = champ_counts.rename(columns={'Champion':'Tm'})
champ_counts['Type']='Champion'
champ_counts


champ_pre_pivot = champ_counts.append(runnerups_counts)

champ_pivot = champ_pre_pivot.pivot_table(values='counts', index=['Tm'], columns=['Type'], aggfunc=np.sum)
champ_pivot.fillna(0, inplace=True) 
nba_champ_flat_df = pd.DataFrame(champ_pivot.to_records())
nba_champ_flat_df['NBA_Finals_Appearances'] = nba_champ_flat_df['Champion'] + nba_champ_flat_df['Runner-Ups']
nba_champ_flat_df = nba_champ_flat_df.sort_values('NBA_Finals_Appearances', ascending=True)
nba_champ_flat_df

,Tm,Champion,Runner-Ups,NBA_Finals_Appearances
0,Baltimore Bullets,0.0,1.0,1.0
19,Philadelphia Warriors,1.0,0.0,1.0
9,Indiana Pacers,0.0,1.0,1.0
22,Rochester Royals,1.0,0.0,1.0
16,Oklahoma City Thunder,0.0,1.0,1.0
28,Toronto Raptors,1.0,0.0,1.0
24,San Francisco Warriors,0.0,2.0,2.0
6,Fort Wayne Pistons,0.0,2.0,2.0
29,Utah Jazz,0.0,2.0,2.0
20,Phoenix Suns,0.0,2.0,2.0


In [61]:
fig = go.Figure()
fig.add_trace(go.Bar(
    y=nba_champ_flat_df['Tm'],
    x=nba_champ_flat_df['Champion'],
    text=nba_champ_flat_df['NBA_Finals_Appearances'],
    #textposition='auto',
    name='Champion',
    orientation='h',
    marker=dict(
        color='rgba(50, 205, 50, 0.6)'
    )
))

fig.add_trace(go.Bar(
    y=nba_champ_flat_df['Tm'],
    x=nba_champ_flat_df['Runner-Ups'],
    name='Runner-Ups',
    orientation='h',
    marker=dict(
        color='rgba(224, 220, 220, 0.6)'
    )
))


#fig.update_layout(barmode='stack')
fig.update_layout(
    xaxis=dict(
        showgrid=True,
        showline=False,
        showticklabels=True,
        
        #zeroline=False,
        #domain=[0.15, 1]
    ),
    yaxis=dict(
    #    showgrid=False,
    #    showline=False,
    #    showticklabels=False,
    #    zeroline=False,
    ),
    barmode='stack',
    paper_bgcolor='rgb(255, 255, 255)',
    plot_bgcolor='rgb(255, 255, 255)',
    #margin=dict(l=120, r=10, t=140, b=80),
    showlegend=True,
    #legend=dict(x=-.15, y=1.155, font_size=10),
    title_text='Teams that have made at least 1 NBA Finals appearance since 1950',
)
fig.show()
# to upload to plotly folder
#py.plot(fig, filename = 'scraping_project_nba_bar1', auto_open=True)

### add team acronym to final participant list for later joins with player stats

In [14]:
df_nba_champs_sh_name = pd.read_csv('champs/champs_runnerups_nba.csv')
df_champs_merge = pd.merge(df_nba_champs_sh_name,nba_champs, how='inner', left_on = 'Tm_Full_Name', right_on = 'Champion')
df_champs_merge = pd.merge(df_nba_champs_sh_name,df_champs_merge, how='right', left_on = 'Tm_Full_Name', right_on = 'Runner-Up')
df_champs_merge = df_champs_merge.rename(columns={"Tm_y": "Champ_tm", "Tm_x": "Runner-Up_tm"})
df_champs_merge = df_champs_merge.sort_values(['Year'])
df_champs_merge

,Runner-Up_tm,Tm_Full_Name_x,Champ_tm,Tm_Full_Name_y,Year,Lg,Champion,Runner-Up,Finals MVP,Points,Rebounds,Assists,Win Shares
54,SYR,Syracuse Nationals,MNL,Minneapolis Lakers,1950,NBA,Minneapolis Lakers,Syracuse Nationals,NaN,G. Mikan (376),NaN,J. Pollard (56),G. Mikan (3.7)
39,NYK,New York Knicks,ROC,Rochester Royals,1951,NBA,Rochester Royals,New York Knicks,NaN,A. Risen (273),A. Risen (196),D. McGuire (78),B. Wanzer (2.7)
38,NYK,New York Knicks,MNL,Minneapolis Lakers,1952,NBA,Minneapolis Lakers,New York Knicks,NaN,G. Mikan (307),G. Mikan (207),D. McGuire (90),G. Mikan (2.4)
37,NYK,New York Knicks,MNL,Minneapolis Lakers,1953,NBA,Minneapolis Lakers,New York Knicks,NaN,G. Mikan (238),G. Mikan (185),D. McGuire (70),G. Mikan (2.0)
53,SYR,Syracuse Nationals,MNL,Minneapolis Lakers,1954,NBA,Minneapolis Lakers,Syracuse Nationals,NaN,G. Mikan (252),G. Mikan (171),P. Seymour (60),G. Mikan (3.5)
58,FTW,Fort Wayne Pistons,SYR,Syracuse Nationals,1955,NBA,Syracuse Nationals,Fort Wayne Pistons,NaN,D. Schayes (209),D. Schayes (141),A. Phillip (78),D. Schayes (1.8)
57,FTW,Fort Wayne Pistons,PHW,Philadelphia Warriors,1956,NBA,Philadelphia Warriors,Fort Wayne Pistons,NaN,P. Arizin (289),N. Johnston (143),G. King (60),P. Arizin (2.2)
52,STL,St. Louis Hawks,BOS,Boston Celtics,1957,NBA,Boston Celtics,St. Louis Hawks,NaN,B. Pettit (298),B. Russell (244),B. Cousy (93),B. Pettit (1.7)
4,BOS,Boston Celtics,STL,St. Louis Hawks,1958,NBA,St. Louis Hawks,Boston Celtics,NaN,C. Hagan (305),B. Russell (221),B. Cousy (82),C. Hagan (2.7)
34,MNL,Minneapolis Lakers,BOS,Boston Celtics,1959,NBA,Boston Celtics,Minneapolis Lakers,NaN,E. Baylor (331),B. Russell (305),B. Cousy (119),F. Ramsey (2.0)


In [15]:
#Advanced stats of champions plus bios
df_rs_adv_champs = pd.merge(df_rs_advanced, 
                            df_champs_merge, 
                            how='inner', 
                            left_on = ['Tm','Season'], 
                            right_on = ['Champ_tm','Year'])
df_rs_adv_champs = pd.merge(df_rs_adv_champs,
                            df_bios,
                            how='inner',
                            left_on = 'Player_ID',
                            right_on = 'Player_ID')
df_rs_adv_champs['RunnerUp_Bool']=False
df_rs_adv_champs['Champ_Bool']=True

#Advanced stats of runner-ups plus bios
df_rs_adv_runnerups = pd.merge(df_rs_advanced, 
                               df_champs_merge, 
                               how='inner', 
                               left_on = ['Tm','Season'], 
                               right_on = ['Runner-Up_tm','Year'])
df_rs_adv_runnerups = pd.merge(df_rs_adv_runnerups,
                            df_bios,
                            how='inner',
                            left_on = 'Player_ID',
                            right_on = 'Player_ID')
df_rs_adv_runnerups['RunnerUp_Bool']=True
df_rs_adv_runnerups['Champ_Bool']=False

### append dataframes to each other

In [16]:
df_rs_adv_finalists = df_rs_adv_champs.append(df_rs_adv_runnerups)
df_rs_adv_finalists.keys()


Index(['Unnamed: 0_x', 'Player_Name_x', 'Player_ID', 'Rk', 'Pos_x', 'Age',
       'Tm', 'G', 'MP', 'PER', 'TS%', '3PAr', 'FTr', 'ORB%', 'DRB%', 'TRB%',
       'AST%', 'STL%', 'BLK%', 'TOV%', 'USG%', 'Unnamed: 19', 'OWS', 'DWS',
       'WS', 'WS/48', 'Unnamed: 24', 'OBPM', 'DBPM', 'BPM', 'VORP', 'Season',
       'Runner-Up_tm', 'Tm_Full_Name_x', 'Champ_tm', 'Tm_Full_Name_y', 'Year',
       'Lg', 'Champion', 'Runner-Up', 'Finals MVP', 'Points', 'Rebounds',
       'Assists', 'Win Shares', 'Unnamed: 0_y', 'Player_Name_y', 'From', 'To',
       'Pos_y', 'Ht', 'Wt', 'Birth Date', 'Colleges', 'Feet', 'Inches',
       'Ht_Inches', 'Ht_cm', 'Ht_m', 'RunnerUp_Bool', 'Champ_Bool'],
      dtype='object')

### select columns to get rid of unnecessary columns

In [17]:
#select column
df_rs_adv_finalists = df_rs_adv_finalists[['Player_ID','Player_Name_x','Pos_x', 'Season',
                                           'Age','Tm','G','MP','PER','TS%', '3PAr', 'FTr',
                                           'OWS', 'DWS', 'WS','WS/48','Ht_Inches','RunnerUp_Bool','Champ_Bool',
                                           'USG%']]
df_rs_adv_finalists
df_rs_adv_finalists.rename({'Player_Name_x':'Player', 'Pos_x':'Pos'}, axis='columns')


    

,Player_ID,Player,Pos,Season,Age,Tm,G,MP,PER,TS%,3PAr,FTr,OWS,DWS,WS,WS/48,Ht_Inches,RunnerUp_Bool,Champ_Bool,USG%
0,carlsdo01,Don Carlson,G-F,1950,30.0,MNL,57,0.0,0.0,0.402,0.000,0.328,0.7,1.9,2.6,0.000,72,False,True,0.0
1,ferriar01,Arnie Ferrin,F-G,1950,24.0,MNL,63,0.0,0.0,0.383,0.000,0.275,0.2,2.3,2.6,0.000,74,False,True,0.0
2,glickno01,Normie Glick,F,1950,22.0,MNL,1,0.0,0.0,1.000,0.000,0.000,0.0,0.0,0.1,0.000,79,False,True,0.0
3,grantbu01,Bud Grant,F,1950,22.0,MNL,35,0.0,0.0,0.371,0.000,0.148,-0.1,0.6,0.5,0.000,75,False,True,0.0
4,harribo02,Bob Harrison,G,1950,22.0,MNL,66,0.0,0.0,0.394,0.000,0.213,0.7,2.7,3.5,0.000,73,False,True,0.0
5,harribo02,Bob Harrison,SG,1952,24.0,MNL,65,1712.0,8.2,0.370,0.000,0.255,-1.0,3.2,2.2,0.061,73,False,True,0.0
6,harribo02,Bob Harrison,SG,1953,25.0,MNL,70,1643.0,10.5,0.421,0.000,0.319,0.6,2.2,2.9,0.084,73,False,True,0.0
7,jarosto01,Tony Jaros,F-G,1950,29.0,MNL,61,0.0,0.0,0.362,0.000,0.332,-0.4,1.7,1.3,0.000,75,False,True,0.0
8,martisl01,Slater Martin*,PG,1950,24.0,MNL,67,0.0,0.0,0.395,0.000,0.308,0.9,2.6,3.5,0.000,70,False,True,0.0
9,martisl01,Slater Martin*,PG,1952,26.0,MNL,66,2480.0,10.8,0.430,0.000,0.301,1.8,4.5,6.3,0.121,70,False,True,0.0


### average height of finals teams

In [18]:
avg_ht_finals_by_season = df_rs_adv_finalists.groupby('Season').mean()
avg_ht_finals_by_season = avg_ht_finals_by_season[['Ht_Inches','3PAr']]
avg_ht_finals_by_season = pd.DataFrame(avg_ht_finals_by_season.to_records())
avg_ht_finals_by_season = avg_ht_finals_by_season.round({'Ht_Inches': 0, '3PAr': 2})

### graph avg ht of finals teams

In [19]:

data = avg_ht_finals_by_season

fig = px.bar(data, x='Season', y='Ht_Inches',
             hover_data=['Ht_Inches', '3PAr'], color='3PAr',
             labels={'Ht_Inches':'Inches'} 
             #height=400
            )
fig.update_layout(xaxis_tickangle=-45,
                  paper_bgcolor='rgb(255, 255, 255)',
                  plot_bgcolor='rgb(255, 255, 255)',
                  title_text='Average Height & 3 Point Attempt Rates of NBA Finals Teams'
                 )
fig.update_yaxes(showgrid=False, 
                 zeroline=False,
                 range=[72, 84])
fig.show()
#py.plot(fig, filename = 'scraping_project_nba_2', auto_open=True)

### Players on finals teams are definitely getting taller on average, as well as taking a higher rate of 3 point shots. However, I think it would be more telling if we could weight each player's height by the % of minutes they accounted for. This would determine the average height of teams based on players who play the most.

In [20]:
# first dataframe is showing frequency of players and sum of Minutes Played (MP) by Season and Height 
df_grp1 = df_rs_adv_finalists.groupby(['Season','Ht_Inches']).agg(['sum','count'])
df_grp1 = pd.DataFrame(df_grp1.to_records())
df_grp1 = df_grp1[["Season","Ht_Inches","('Player_ID', 'count')","('MP', 'sum')"]]
df_grp1 = df_grp1.rename(columns={"('Player_ID', 'count')":"Freq_Ht_Season", "('MP', 'sum')":"MP_Ht_Season"})

# second dataframe is subset with frequency of players and sum of MP by just Season
df_grp2 = df_rs_adv_finalists.groupby('Season').agg(['sum','count'])
df_grp2 = pd.DataFrame(df_grp2.to_records())
df_grp2 = df_grp2[["Season","('Player_ID', 'count')","('MP', 'sum')"]]
df_grp2 = df_grp2.rename(columns={"('Player_ID', 'count')":"Freq_Season_Tot", "('MP', 'sum')":"MP_Season_Tot"})

# merging dataframes above
df_grp_m = pd.merge(df_grp1, df_grp2, how='inner', on='Season')
df_grp_m

# calculate weighted average count by height (inches) by dividing total MP per height class by total season MP
df_grp_m['MP_Ht%Season'] = df_grp_m['MP_Ht_Season']/df_grp_m['MP_Season_Tot']
df_grp_m['Weighted_avg_ct_by_Ht'] = df_grp_m['MP_Ht%Season']*df_grp_m['Ht_Inches']  

# filtering out seasons 1950 - 1952 since MP stat wasn't tracked
df_grp_m = df_grp_m[df_grp_m.Season > 1952]

# grouping by season and summing weighted average count by height to find average height per season weighted by MP
df_grp_m_4bargraph2 = df_grp_m[['Season','Weighted_avg_ct_by_Ht']].groupby('Season').sum()

# joining previous dataset to add avg 3pt shot
df_grp_m_4bargraph2 = pd.merge(df_grp_m_4bargraph2,
                               avg_ht_finals_by_season,
                               how="inner",
                               on="Season"
                              )
                               
df_grp_m_4bargraph2 = df_grp_m_4bargraph2.round({'Weighted_avg_ct_by_Ht': 0, '3PAr': 2})
df_grp_m_4bargraph2 = df_grp_m_4bargraph2.rename(columns={'Ht_Inches':'Avg_Ht_Inches','Weighted_avg_ct_by_Ht':'Avg_Ht_Inches_wt_MP'})
df_grp_m_4bargraph2
    
# subsets for historgram showing frequency of final players by height
#df_grp_m_4histo = df


,Season,Avg_Ht_Inches_wt_MP,Avg_Ht_Inches,3PAr
0,1953,76.0,76.0,0.00
1,1954,76.0,76.0,0.00
2,1955,77.0,77.0,0.00
3,1956,77.0,77.0,0.00
4,1957,77.0,77.0,0.00
5,1958,77.0,77.0,0.00
6,1959,77.0,77.0,0.00
7,1960,77.0,76.0,0.00
8,1961,77.0,77.0,0.00
9,1962,77.0,77.0,0.00


### graph showing avg height of finals team weighted by minutes played

In [21]:

data = df_grp_m_4bargraph2

fig = px.bar(data, x='Season', y='Avg_Ht_Inches_wt_MP',
             hover_data=['Avg_Ht_Inches_wt_MP', '3PAr'], color='3PAr',
             labels={'Avg_Ht_Inches_wt_MP':'Inches'} 
             #height=400
            )
fig.update_layout(xaxis_tickangle=-45,
                  paper_bgcolor='rgb(255, 255, 255)',
                  plot_bgcolor='rgb(255, 255, 255)',
                  title_text='Average Height Weighted by Minutes Played & 3 Point Attempt Rates of NBA Finals Teams'
                 )
fig.update_yaxes(showgrid=False, 
                 zeroline=False,
                 range=[72, 84])
fig.show()

#py.plot(fig, filename = 'scraping_project_nba_3', auto_open=True)

### Not much of a difference when weighting by minutes played. Let's weight by a stat called Win Shares instead

#### explanation of Win Shares can be found here: https://www.basketball-reference.com/about/ws.html

In [22]:
# first dataframe is showing frequency of players and sum of Win Shares (WS) by Season and Height 
df_grp1_WS = df_rs_adv_finalists.groupby(['Season','Ht_Inches']).agg(['sum','count'])
df_grp1_WS = pd.DataFrame(df_grp1_WS.to_records())
df_grp1_WS = df_grp1_WS[["Season","Ht_Inches","('Player_ID', 'count')","('WS', 'sum')"]]
df_grp1_WS = df_grp1_WS.rename(columns={"('Player_ID', 'count')":"Freq_Ht_Season", "('WS', 'sum')":"WS_Ht_Season"})

# second dataframe is subset with frequency of players and sum of WS by just Season
df_grp2_WS = df_rs_adv_finalists.groupby('Season').agg(['sum','count'])
df_grp2_WS = pd.DataFrame(df_grp2_WS.to_records())
df_grp2_WS = df_grp2_WS[["Season","('Player_ID', 'count')","('WS', 'sum')"]]
df_grp2_WS = df_grp2_WS.rename(columns={"('Player_ID', 'count')":"Freq_Season_Tot", "('WS', 'sum')":"WS_Season_Tot"})

# merging dataframes above
df_grp_m_WS = pd.merge(df_grp1_WS, df_grp2_WS, how='inner', on='Season')
df_grp_m_WS

# calculate weighted average count by height (inches) by dividing total MP per height class by total season MP
df_grp_m_WS['WS_Ht%Season'] = df_grp_m_WS['WS_Ht_Season']/df_grp_m_WS['WS_Season_Tot']
df_grp_m_WS['Weighted_avg_ct_by_Ht'] = df_grp_m_WS['WS_Ht%Season']*df_grp_m_WS['Ht_Inches']  

# filtering out seasons 1950 - 1952 since MP stat wasn't tracked
df_grp_m_WS = df_grp_m_WS[df_grp_m_WS.Season > 1952]

# grouping by season and summing weighted average count by height to find average height per season weighted by MP
df_grp_m_WS4bargraph2 = df_grp_m_WS[['Season','Weighted_avg_ct_by_Ht']].groupby('Season').sum()

# joining previous dataset to add avg 3pt shot
df_grp_m_WS4bargraph2 = pd.merge(df_grp_m_WS4bargraph2,
                               avg_ht_finals_by_season,
                               how="inner",
                               on="Season"
                              )
                               
df_grp_m_WS4bargraph2 = df_grp_m_WS4bargraph2.round({'Weighted_avg_ct_by_Ht': 0, '3PAr': 2})
df_grp_m_WS4bargraph2 = df_grp_m_WS4bargraph2.rename(columns={'Ht_Inches':'Avg_Ht_Inches','Weighted_avg_ct_by_Ht':'Avg_Ht_Inches_wt_WS'})
df_grp_m_WS4bargraph2
    
# subsets for historgram showing frequency of final players by height
#df_grp_m_4histo = df


,Season,Avg_Ht_Inches_wt_WS,Avg_Ht_Inches,3PAr
0,1953,77.0,76.0,0.00
1,1954,77.0,76.0,0.00
2,1955,78.0,77.0,0.00
3,1956,78.0,77.0,0.00
4,1957,78.0,77.0,0.00
5,1958,78.0,77.0,0.00
6,1959,77.0,77.0,0.00
7,1960,78.0,76.0,0.00
8,1961,78.0,77.0,0.00
9,1962,77.0,77.0,0.00


In [23]:
data = df_grp_m_WS4bargraph2

fig = px.bar(data, x='Season', y='Avg_Ht_Inches_wt_WS',
             hover_data=['Avg_Ht_Inches_wt_WS', '3PAr'], color='3PAr',
             labels={'Avg_Ht_Inches_wt_WS':'Inches'} 
             #height=400
            )
fig.update_layout(xaxis_tickangle=-45,
                  paper_bgcolor='rgb(255, 255, 255)',
                  plot_bgcolor='rgb(255, 255, 255)',
                  title_text='Average Height Weighted by Win Shares (WS) & 3 Point Attempt Rates of NBA Finals Teams'
                 )
fig.update_yaxes(showgrid=False, 
                 zeroline=False,
                 range=[72, 84])
fig.show()

#py.plot(fig, filename = 'scraping_project_nba_4', auto_open=True)

In [29]:
df_rs_adv_finalists = df_rs_adv_finalists.sort_values(['Season'])
df_rs_adv_finalists['Team_Results'] = np.where(df_rs_adv_finalists['Champ_Bool']==True, 'Champ', 'Lost')
df_rs_adv_finalists

,Player_ID,Player_Name_x,Pos_x,Season,Age,Tm,G,MP,PER,TS%,...,FTr,OWS,DWS,WS,WS/48,Ht_Inches,RunnerUp_Bool,Champ_Bool,USG%,Team_Results
0,carlsdo01,Don Carlson,G-F,1950,30.0,MNL,57,0.0,0.0,0.402,...,0.328,0.7,1.9,2.6,0.000,72,False,True,0.0,Champ
25,schaehe01,Herm Schaefer,G-F,1950,31.0,MNL,65,0.0,0.0,0.460,...,0.322,2.9,2.4,5.4,0.000,72,False,True,0.0,Champ
6,levanan01,Andrew Levane,F-G,1950,29.0,SYR,60,0.0,0.0,0.365,...,0.203,-0.2,2.0,1.8,0.000,74,True,False,0.0,Lost
7,macknjo01,Johnny Macknowski,G-F,1950,27.0,SYR,59,0.0,0.0,0.405,...,0.384,0.9,1.8,2.8,0.000,72,True,False,0.0,Lost
8,petered01,Ed Peterson,C,1950,25.0,SYR,62,0.0,0.0,0.472,...,0.474,3.0,2.1,5.1,0.000,81,True,False,0.0,Lost
9,ratkoge01,George Ratkovicz,PF,1950,27.0,SYR,62,0.0,0.0,0.452,...,0.793,3.4,2.6,6.0,0.000,78,True,False,0.0,Lost
12,seymopa01,Paul Seymour,G-F,1950,22.0,SYR,62,0.0,0.0,0.396,...,0.336,1.1,2.7,3.8,0.000,73,True,False,0.0,Lost
2,corlera01,Ray Corley,G,1950,22.0,SYR,60,0.0,0.0,0.365,...,0.330,-0.3,1.6,1.2,0.000,72,True,False,0.0,Lost
1,chollle01,Leroy Chollet,F,1950,24.0,SYR,49,0.0,0.0,0.385,...,0.313,0.1,0.8,0.9,0.000,74,True,False,0.0,Lost
5,hannual01,Alex Hannum*,PF,1950,26.0,SYR,64,0.0,0.0,0.423,...,0.381,2.0,3.2,5.1,0.000,79,True,False,0.0,Lost


In [59]:
fig = px.scatter(df_rs_adv_finalists[df_rs_adv_finalists.Season >= 1953],
                 y="Ht_Inches", 
                 x="WS", 
                 size="TS%", 
                 color="Team_Results",
                 #color_continuous_scale=px.colors.qualitative.Antique,
                 #log_x=True, 
                 #size_max=20.0,
                 template='plotly_white',
                 hover_data=['Player_Name_x','Tm','Season', '3PAr'],
                 title="Finals Players Relationship Between Win Shares (WS) and Height (inches)",
                 animation_frame="Season", #animation_group="Pos_x"
                 )

fig.update_layout(paper_bgcolor='rgb(255, 255, 255)',
                  plot_bgcolor='rgb(255, 255, 255)'
                 )
fig.show()

#py.plot(fig, filename = 'scraping_project_nba_5', auto_open=True)

'https://plot.ly/~davila2345/27/'

### Producing Top 20 WS performances of all time. Note that most players are bigs and wings. Only Steph Curry and Jerry West make list as players under 75 inches (6'3")

In [331]:
top_20_ws = df_rs_adv_finalists[df_rs_adv_finalists.Season >= 1953].sort_values(by='WS', ascending=False).head(20)
 

fig = go.Figure(data=[go.Table(
    header=dict(values=list(['Season', 
                       'Player', 
                       'Tm', 
                       'Pos', 
                       'Age', 
                       'Ht (In.)', 
                       'MP',
                       'WS']),
                fill_color='gray',
                font_color='white',
                align='left'),
    cells=dict(values=[top_20_ws.Season, 
                       top_20_ws.Player_Name_x, 
                       top_20_ws.Tm, 
                       top_20_ws.Pos_x, 
                       top_20_ws.Age, 
                       top_20_ws.Ht_Inches, 
                       top_20_ws.MP,
                       top_20_ws.WS,],
               line_color='gray',
               fill_color='white',
               align='left'))
])

fig.update_layout(title='Top 20 NBA Win Shares (WS) of All-Time')
fig.show()
#py.plot(fig, filename = 'scraping_project_nba_tbl1', auto_open=True)

### Distribution of top 100 WS players by height

In [348]:
top_100_ws = df_rs_adv_finalists[df_rs_adv_finalists.Season >= 1953].sort_values(by='WS', ascending=False).head(100)
x = top_100_ws
x

fig = px.histogram(x, x="Ht_Inches")
fig.update_layout(title='Top 100 NBA Win Shares (WS) of All-Time',
                  paper_bgcolor='rgb(255, 255, 255)',
                  plot_bgcolor='rgb(255, 255, 255)')
fig.show()
#py.plot(fig, filename = 'nba_top100_ws_histo', auto_open=True)

'https://plot.ly/~davila2345/31/'